# Top Brass Trophy problem (Ex. 5.1 in Rardin'98)

Top Brass Trophy Company makes large championship trophies for youth athletic leagues. At the moment, they are planning production for fall sports: football and soccer. Each football trophy has a wood base, an engraved plaque, a large brass football on top, and returns $\$ 12$ in profit. Soccer trophies are similar except that a brass soccer ball is on top, and the unit profit is only $\$ 9$. Since the football has an asymmetric shape, its base requires 4 board feet of wood; the soccer base requires only 2 board feet. At the moment there are 1000 brass footballs in stock, 1500 soccer balls, 1750 plaques, and 4800 board feet of wood. What trophies should be produced from these supplies to maximize total profit assuming that all that are made can be sold?

### Minimal solution

In [1]:
using JuMP, Clp

m = Model(with_optimizer(Clp.Optimizer))
@variable(m, f >= 0)                        # football trophies
@variable(m, s >= 0)                        # soccer trophies
@constraint(m, Cwood,   4f + 2s <= 4800)    # total board feet of wood
@constraint(m, Cplaques,  f + s <= 1750)    # total number of plaques
@constraint(m, Cfballs,       f <= 1000)    # total number of brass footballs
@constraint(m, Csballs,       s <= 1500)    # total number of brass soccer balls
@objective(m, Max, 12f + 9s)                # maximize profit

optimize!(m)

display(m)

println(termination_status(m))
println("Build ", value(f), " football trophies.")
println("Build ", value(s), " soccer trophies.")
println("Total profit will be \$", objective_value(m))

A JuMP Model
Maximization problem with:
Variables: 2
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Clp
Names registered in the model: Cfballs, Cplaques, Csballs, Cwood, f, s

OPTIMAL
Build 650.0 football trophies.
Build 1100.0 soccer trophies.
Total profit will be $17700.0
Coin0506I Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
Clp0006I 0  Obj -0 Dual inf 20.999998 (2)
Clp0006I 2  Obj 17700
Clp0000I Optimal - objective value 17700
Coin0511I After Postsolve, objective 17700, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 17700 - 2 iterations time 0.002, Presolve 0.00


### Dual variables

In [2]:
# Shadow price (change in objective from a change in the right hand side of the constraint)
println("Shadow price for wood: ", shadow_price(Cwood))
println("Shadow price for plaques: ", shadow_price(Cplaques))
println("Shadow price for brass footballs: ", shadow_price(Cfballs))
println("Shadow price for brass soccer balls: ", shadow_price(Csballs))

println()

# # Dual variables (as provided by the solver) 
# println("Dual variable for wood: ", dual(Cwood))
# println("Dual variable for plaques: ", dual(Cplaques))
# println("Dual variable for brass footballs: ", dual(Cfballs))
# println("Dual variable for brass soccer balls: ", dual(Csballs))

Shadow price for wood: 1.5
Shadow price for plaques: 6.0
Shadow price for brass footballs: -0.0
Shadow price for brass soccer balls: -0.0



### Dual problem

In [3]:
using JuMP, Clp

m_dual = Model(with_optimizer(Clp.Optimizer))
@variable(m_dual, λ[1:4] >= 0)
@constraint(m_dual, 4λ[1] + λ[2] + λ[3] >= 12)
@constraint(m_dual, 2λ[1] + λ[2] + λ[4] >= 9)
@objective(m_dual, Min, 4800λ[1] + 1750λ[2] + 1000λ[3] + 1500λ[4])

optimize!(m_dual)

display(m_dual)

println(termination_status(m_dual))
println("dual variables are: ", value.(λ))
println("Optimal objective is: ", objective_value(m))



A JuMP Model
Minimization problem with:
Variables: 4
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Clp
Names registered in the model: λ

OPTIMAL
dual variables are: [1.5, 6.0, 0.0, 0.0]
Optimal objective is: 17700.0
Coin0506I Presolve 2 (0) rows, 4 (0) columns and 6 (0) elements
Clp0006I 0  Obj 0 Primal inf 7.4999998 (2)
Clp0006I 3  Obj 17700
Clp0000I Optimal - objective value 17700
Clp0032I Optimal objective 17700 - 3 iterations time 0.002
